[View in Colaboratory](https://colab.research.google.com/github/nenorden/Transfer-learning-on-Fashion-MNIST/blob/master/LogisticRegression.ipynb)

# Download Fashion-MNIST

In [1]:
import os
os.chdir('/content/')
!git clone https://github.com/zalandoresearch/fashion-mnist.git 
os.chdir('/content/fashion-mnist/')
import utils.mnist_reader as mnist_reader
os.chdir("/")
train_x_orig, train_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='train')
test_x_orig, test_y = mnist_reader.load_mnist('content/fashion-mnist/data/fashion', kind='t10k')


Cloning into 'fashion-mnist'...
remote: Counting objects: 615, done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 615 (delta 5), reused 9 (delta 4), pack-reused 603
Receiving objects: 100% (615/615), 105.18 MiB | 50.40 MiB/s, done.
Resolving deltas: 100% (347/347), done.


# Import ResNet Features

## Download features from Drive to virtual machine

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

train_file_id = '1j1488T_da1oVlwwy-_19GMuY9tSucqzg'
test_file_id = '1qR7fOsQ6TTW8eV8SnKN-jVvswlzMx9QM'

os.chdir('/content/')
# Load files by ID
downloaded = drive.CreateFile({'id': train_file_id})
downloaded.GetContentFile('train_fashion-MNIST_resnet.npy')

downloaded = drive.CreateFile({'id': test_file_id})
downloaded.GetContentFile('test_fashion-MNIST_resnet.npy')


# Files should be in /content/. Check!

!ls
os.chdir('/')


datalab        test_fashion-MNIST_resnet.npy
fashion-mnist  train_fashion-MNIST_resnet.npy


## Load ResNet data

In [0]:
%matplotlib inline
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

os.chdir('/content/')
train_x = np.load('train_fashion-MNIST_resnet.npy')
test_x = np.load('test_fashion-MNIST_resnet.npy')
os.chdir('/')


# Logistic Regression


## ResNet features

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import time

# Hyperparameters. C regularization strength
param_grid = {'C': [1e-1, 1e0], 'max_iter': [100, 200]}

# Create model, fit to training data
# Perform grid search CV to find best hyperparameters
start_time = time.time()
logreg = LogisticRegression()
logreg = GridSearchCV(logreg, param_grid)
logreg.fit(train_x, train_y)

print("--- %s seconds ---" % (time.time() - start_time))

# Predict on test data
logreg_predict = logreg.predict(test_x)

# Get accuracy
logreg_acc = (logreg_predict==test_y).mean()
print(logreg_acc)

--- 2336.4096279144287 seconds ---
0.7771


In [6]:
# Print model with chosen parameters
print(logreg)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.1, 1.0, 5.0], 'max_iter': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


## Original features

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import time

# Hyper parameters. C regularization strength.
param_grid = {'C': [1e-1, 1e0], 'max_iter': [100, 200]}

# Create model, fit to training data.
# Perform grid search CV to find best hyperparameters.
start_time = time.time()
logreg_orig = LogisticRegression()
logreg_orig = GridSearchCV(logreg_orig, param_grid)
logreg_orig.fit(train_x_orig, train_y)

print("--- %s seconds ---" % (time.time() - start_time))

# Print model with chosen hyperparameters
print(logreg_orig)

# Predict on test data
logreg_predict_orig = logreg_orig.predict(test_x_orig)

# Get accuracy
logreg_acc_orig = (logreg_predict_orig==test_y).mean()
print(logreg_acc_orig)